In [1]:
import os
import re
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import seaborn as sns
from matplotlib.pyplot import rc_context
from tqdm.notebook import tqdm, trange
from sklearn.preprocessing import MinMaxScaler
sc.settings.verbosity = 3

In [2]:
p_dir = (Path().cwd().parents[0]).absolute()
data_dir = p_dir / "data"

In [3]:
%load_ext autoreload
%autoreload 2

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)


In [4]:
import scSpaMet as spamet

In [5]:
experiment = "Lung"
cores = ['B5', 'C6', 'D4', 'E4', 'E6', 'F4']
datasets = [f'{core}_{i}' for core in cores for i in range(1,5)] + ['F7_3', 'F7_4']

In [6]:
# experiment = "tonsil"
# cores = ["DonorA", "DonorE"]
# datasets = [core + f"_{i}" for i in range(1, 7) for core in cores]

# Load data

In [7]:
def read_props(dataset, experiment):
    df_morph = pd.read_csv(data_dir / "props" / f"morphology_IMC_{experiment}_{dataset}.csv")
    df_intensity_IMC = pd.read_csv(data_dir / "props" / f"intensity_IMC_{experiment}_{dataset}.csv")
    df_intensity_TS = pd.read_csv(data_dir / "props" / f"intensity_TS_{experiment}_{dataset}_auto.csv")

    return df_morph, df_intensity_IMC, df_intensity_TS

In [8]:
%%capture 

adatas = []
adatas_raw = []
df_sums = []
for dataset in datasets:
    try:
        df_morph, df_intensity_IMC, df_intensity_TS = read_props(dataset, experiment)
        print(f'Sucessfully read dataset {dataset}') 
    except:
        print(f'{dataset} cannot be read') 
        continue
    try:
        df_intensity_TS.drop(['Rest', 'Total'], axis=1, inplace=True)
    except:
        pass
    df_intensity_TS.iloc[:, 1:] = df_intensity_TS.iloc[:, 1:].multiply(
        df_morph["area"], axis=0
    )
    # data_all = df_intensity_TS.iloc[:, 1:].values
    # data_all_norm = (data_all+0.1)/(np.percentile(data_all,50,axis=1,keepdims=True)+0.1)
    # data_all_norm = MinMaxScaler().fit_transform(data_all)
    # df_intensity_TS.iloc[:, 1:] = data_all_norm
    
    # Merge TS and IMC data
    df = df_intensity_TS
    df.set_index("Id", inplace=True)
    df_sums.append(df.sum(axis=0))
    
    # Put to adata format
    adata = sc.AnnData(df.values)
    adata.var_names = df.columns.tolist()
    adata.obs["Cell"] = df.index.tolist()
    adata.obs["Dataset"] = dataset
    adata.obs["Core"] = dataset.split("_")[0]
    adata.obsm["spatial"] = df_morph[["centroid-0", "centroid-1"]].to_numpy()
    
    # FPM normalize
    sc.pp.normalize_total(adata, target_sum=1e5)
    # sc.pp.log1p(adata, base=2)
    adatas_raw.append(adata.copy())
    
    # Standard scale
    sc.pp.scale(adata)
    adatas.append(adata)
    
adata = ad.concat(adatas, join="inner")
adata_raw = ad.concat(adatas_raw, join="inner")

normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
   

In [9]:
# Read Protein clustering info
path = data_dir / "adata" / f"{experiment}.h5ad"
adata_IMC_cluster = ad.read_h5ad(path)


C:\Users\thu71\Anaconda3\envs\scanpy\Lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [10]:
adata = adata[~adata.obs.Dataset.isin(['F7_1', 'F7_2'])]
adata_IMC = adata_IMC_cluster[~adata_IMC_cluster.obs.Dataset.isin(['F7_1', 'F7_2'])]

In [11]:
df_sum = pd.concat(df_sums, axis=1).mean(axis=1)
# mz_qc = df_sum[(df_sum > 16000)].index.tolist()
mz_qc = df_sum[(df_sum > 220)].index.tolist()

In [12]:
len(mz_qc)

197

In [13]:
adata = adata[:, mz_qc]
adata_raw = adata_raw[:, mz_qc]

In [14]:
# Define IMC and SIMS marker list
IMC_markers = adata.var_names.tolist()
SIMS_masses = adata_IMC.var_names.tolist()

# Joint embedding

In [15]:
def Umap(a,rep=None):
    if rep is not None:
        print(f'Using representation {rep}')
        sc.pp.neighbors(a,use_rep=rep, metric='cosine')
    else:
        sc.pp.neighbors(a)
    sc.tl.umap(a)
    print('Sucessfully run Umap!')
    return a

def TSNE(a,rep=None):
    if rep is not None:
        print(f'Using representation {rep}')
        sc.tl.tsne(a,use_rep=rep)
    else:
        sc.tl.tsne(a)
    print('Sucessfully run TSNE!')
    return a

In [16]:
train_x_IMC = adata_IMC.X
train_x_SIMS = adata.X

In [17]:
train_x_IMC.shape

(19507, 12)

In [18]:
train_x_SIMS.shape

(19507, 197)

In [19]:
# X_embedding, _, _ = spamet.tl.Joint_XVAE_clustering(train_x_IMC, train_x_SIMS, epochs=20)
X_embedding, q, X_predict = spamet.tl.Joint_XVAE_clustering(train_x_IMC, train_x_SIMS, epochs=50, resolution=0.3, latent_dim=16, netwidths_1=[16,8,8], netwidths_2=[128,64,32,16])

GPU name:  []


C:\Users\thu71\Anaconda3\envs\scanpy\Lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch 1/50
610/610 - 5s - loss: 0.5161 - reconstruction_loss: 0.5073 - kl_loss: 0.8796 - 5s/epoch - 8ms/step
Epoch 2/50
610/610 - 2s - loss: 0.4304 - reconstruction_loss: 0.4262 - kl_loss: 0.4114 - 2s/epoch - 2ms/step
Epoch 3/50
610/610 - 1s - loss: 0.3853 - reconstruction_loss: 0.3800 - kl_loss: 0.5280 - 1s/epoch - 2ms/step
Epoch 4/50
610/610 - 1s - loss: 0.3573 - reconstruction_loss: 0.3509 - kl_loss: 0.6368 - 1s/epoch - 2ms/step
Epoch 5/50
610/610 - 2s - loss: 0.3399 - reconstruction_loss: 0.3329 - kl_loss: 0.7069 - 2s/epoch - 3ms/step
Epoch 6/50
610/610 - 2s - loss: 0.3263 - reconstruction_loss: 0.3189 - kl_loss: 0.7466 - 2s/epoch - 3ms/step
Epoch 7/50
610/610 - 2s - loss: 0.3173 - reconstruction_loss: 0.3096 - kl_loss: 0.7753 - 2s/epoch - 2ms/step
Epoch 8/50
610/610 - 2s - loss: 0.3095 - reconstruction_loss: 0.3015 - kl_loss: 0.8027 - 2s/epoch - 3ms/step
Epoch 9/50
610/610 - 2s - loss: 0.3027 - reconstruction_loss: 0.2945 - kl_loss: 0.8177 - 2s/epoch - 3ms/step
Epoch 10/50
610/610

C:\Users\thu71\Anaconda3\envs\scanpy\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\thu71\Anaconda3\envs\scanpy\Lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\thu71\Anaconda3\envs\scanpy\Lib\site-packages\umap\distances.py:1086: NumbaDepr

    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:33)
running Leiden clustering
    finished: found 7 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:03)
The value of delta_label of current 1 th iteration is 0.18644589121853694 >= tol 0.005
Epoch 1/50
610/610 [==============================] - 3s 2ms/step - loss: 0.0703
Epoch 2/50
610/610 [==============================] - 1s 2ms/step - loss: 0.0284
Epoch 3/50
610/610 [==============================] - 1s 2ms/step - loss: 0.0239
Epoch 4/50
610/610 [==============================] - 1s 2ms/step - loss: 0.0215
Epoch 5/50
610/610 [==============================] - 1s 2ms/step - loss: 0.0201
Epoch 6/50
610/610 [==============================] - 1s 2ms/step - loss: 0.0190
Epoch 7/50
610/610 [==============================] - 1s 2ms/step - loss: 0.0186
Epoch 8/50
610/610 [==================

In [20]:
_, X_predict_IMC = spamet.tl.Joint_VAE(train_x_IMC, epochs=50,  netwidths=[16,8], latent_dim=8)

_, X_predict_SIMS = spamet.tl.Joint_VAE(train_x_SIMS, epochs=50,  netwidths=[128,64,32,16], latent_dim=8)

GPU name:  []


C:\Users\thu71\Anaconda3\envs\scanpy\Lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch 1/50
610/610 - 3s - loss: 0.0698 - reconstruction_loss: 0.0653 - kl_loss: 0.0449 - 3s/epoch - 4ms/step
Epoch 2/50
610/610 - 1s - loss: 0.0328 - reconstruction_loss: 0.0300 - kl_loss: 0.0279 - 806ms/epoch - 1ms/step
Epoch 3/50
610/610 - 1s - loss: 0.0264 - reconstruction_loss: 0.0226 - kl_loss: 0.0380 - 826ms/epoch - 1ms/step
Epoch 4/50
610/610 - 1s - loss: 0.0255 - reconstruction_loss: 0.0218 - kl_loss: 0.0372 - 840ms/epoch - 1ms/step
Epoch 5/50
610/610 - 1s - loss: 0.0252 - reconstruction_loss: 0.0214 - kl_loss: 0.0382 - 805ms/epoch - 1ms/step
Epoch 6/50
610/610 - 1s - loss: 0.0244 - reconstruction_loss: 0.0201 - kl_loss: 0.0421 - 805ms/epoch - 1ms/step
Epoch 7/50
610/610 - 1s - loss: 0.0226 - reconstruction_loss: 0.0179 - kl_loss: 0.0472 - 806ms/epoch - 1ms/step
Epoch 8/50
610/610 - 1s - loss: 0.0222 - reconstruction_loss: 0.0174 - kl_loss: 0.0486 - 840ms/epoch - 1ms/step
Epoch 9/50
610/610 - 1s - loss: 0.0219 - reconstruction_loss: 0.0170 - kl_loss: 0.0488 - 850ms/epoch - 1ms/

In [21]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

r_IMC = mean_absolute_error(train_x_IMC, X_predict[0])
r_SIMS = mean_absolute_error(train_x_SIMS, X_predict[1])
print(r_IMC, r_SIMS)

0.082159735 0.5074453301503237


In [22]:
r_IMC = mean_absolute_error(train_x_IMC, X_predict_IMC)
r_SIMS = mean_absolute_error(train_x_SIMS, X_predict_SIMS)
print(r_IMC, r_SIMS)

0.08015019 0.5588502001494667


In [23]:
# from sklearn.metrics import mean_squared_error

# r_IMC = mean_squared_error(train_x_IMC, X_predict_IMC)
# r_SIMS = mean_squared_error(train_x_SIMS, X_predict_SIMS)
# print(r_IMC, r_SIMS)

In [24]:
import pandas as pd

print(pd.__version__)

2.0.2
